In [ ]:
import os
os.environ["HF_HOME"] = "E:/huggingface"
#import transformers
from datasets import load_dataset

dataset = load_dataset("squad_it", trust_remote_code=True)
DEVICE = "cuda:0"

In [ ]:
from get_model import get_model
model_name = "gemma"

model, tokenizer = get_model(model_name)
#model = model.to(DEVICE) #togliere se in 4 bit

In [ ]:
from get_prompt import get_prompt
# del generate_prompt
generate_prompt, stop = get_prompt(model_name) #stop sembra useless

In [ ]:
def build_question(context, question):
    return f"Dato il seguente testo:\n{context}\nRispondi brevemente a questa domanda:\n{question}"

def build_answer(answer):
    return f"Risposta breve: {answer}"

In [ ]:
## test per gemma
def build_question(context, question):
    return f"contesto:\n{context}\ndomanda:\n{question}"

def build_answer(answer):
    return f"risposta: {answer}"

In [ ]:
import random

def get_shots(dataset, n):
    conversation = []
    for i in range(n):
        elem = random.choice(dataset["train"])
        q_shot = build_question(elem["context"], elem["question"])
        conversation.append(dict(
            role="user",
            text=q_shot
        ))
        conversation.append(dict(
            role="assistant",
            text=build_answer(elem["answers"]["text"][0])
        ))
    return conversation

In [ ]:
chat = [
  #{"role": "system", "content": "Sei un assistente utile." },
  {"role": "user", "content": "Ciao come va amico?" },
  {"role": "assistant", "content": "Tutto bene grazie." },
  {"role": "user", "content": "E tua sorella Alice?" },
  {"role": "assistant", "content": "è morta"},
  {"role": "user", "content": "Ah cavolo, mi dispiace."},
  {"role": "assistant", "content": "tranqui bro, è la vita"},
  {"role": "user", "content": "scusa come si chiama tua sorella?"},
] 
print(tokenizer.apply_chat_template(chat, tokenize=False))

In [23]:
import torch

def forward_model(*, prompt, model, tokenizer):

    if "pad_token" not in tokenizer.special_tokens_map:
        tokenizer.pad_token = tokenizer.eos_token

    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        padding=True,
    ).input_ids
    with torch.no_grad():
        input_ids = input_ids.to(DEVICE)
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=32,
            temperature=0.001,
            repetition_penalty=1.2,
            eos_token_id=107,
            do_sample=True,
            num_return_sequences=1
        )
        print(output_ids)

    ret = []
    for i in range(0, len(output_ids)):
        generated_text = tokenizer.decode(
            output_ids[i],
            skip_special_tokens=True
        )
        # print(generated_text)
        # print(generated_text[len(prompt[i]):])
        generated_text = generated_text[len(prompt[i]):] #perchè in output_ids c'è anche il prompt!

        # if "\n" in generated_text:
        #     generated_text = generated_text[:generated_text.index("\n")] non necessario
        
        ret.append(generated_text.strip())

    return ret

In [24]:
import time

t0 = time.time()
out = forward_model(
    prompt=["contesto: Io lavoro a Londra. domanda: Dove lavoro? risposta:"],
    model=model,
    tokenizer=tokenizer,
)
print(time.time() - t0)

print(out)

tensor([[     2,   1739,  18470, 235292,  27266,  35212,    476, 134574, 235265,
          95962, 235292,  70274,  35212, 235336,  76742, 235292,    586, 134574,
         235265, 235248,    107]], device='cuda:0')
0.7209978103637695
['A Londra.']


In [21]:
### CELLA DI PROVA DEL NUOVO GET PROMPT DI SAIGA-ITA-7B!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import uuid
from tqdm import tqdm

batch_size = 1

predicted_answers = []
theoretical_answers = []

ds = dataset["test"].shuffle(seed=42)

total = 0
for i in range(3):
    elem     = ds[i].copy()
    ids      = elem["id"]
    context  = elem["context"]
    question = elem["question"]
    answers  = elem["answers"]

    model_inputs = []
    chat = get_shots(dataset, 3) + [
                dict(
                    role="user",
                    text=build_question(context, question)
                )
            ]
    model_inputs.append(
        generate_prompt(
        # scrive tre esempi non come  prompt ma come cronologia di chat: role: user -> text: domanda, role: assistant -> text: risposta breve
            chat,
            do_continue=False,
        ) + "" + build_answer("").strip()
    )

print(model_inputs[0])

<bos><start_of_turn>user
Di seguito ti verrà fornito un contesto e poi una domanda. il tuo compito è quello di rispondere alla domanda basandoti esclusivamente sul contesto.
contesto:
Napoleone sposò Joséphine de Beauharnais nel 1796, quando aveva 26 anni; era una vedova di 32 anni il cui primo marito era stato giustiziato durante la Rivoluzione. Fino a quando non incontrò Bonaparte, era stata conosciuta come "Rosa", un nome che non gli piaceva. La chiamò invece "Joséphine", e d' ora in poi andò con questo nome. Bonaparte spesso inviato le sue lettere d' amore durante le sue campagne. Ha formalmente adottato suo figlio Eugène e suo cugino Stéphanie e ha organizzato matrimoni dinastici per loro. Joséphine fece sposare la figlia Hortense con il fratello di Napoleone Louis.
domanda:
Chi ha sposato Napoleone nel 1796?
risposta: Joséphine de Beauharnais<end_of_turn>
<start_of_turn>contesto:
La popolarità del mandolino negli Stati Uniti è stata stimolata dal successo di un gruppo di giovani 

In [25]:
import uuid
from tqdm import tqdm

batch_size = 1

predicted_answers = []
theoretical_answers = []

ds = dataset["test"].shuffle(seed=42)
max_len = len(ds)


bar = tqdm(ds.iter(batch_size=batch_size), total=len(ds))
total = 0
for elem in bar:
    try:
        ids = elem["id"]
        context = elem["context"]
        question = elem["question"]
        answers = elem["answers"]

        model_inputs = []
        for c, q in zip(context, question):
            model_inputs.append(
                generate_prompt(
                    get_shots(dataset, 3) + [
                        dict(
                            role="user",
                            text=build_question(c, q)
                        )
                    ],
                    do_continue=False
                ) + " " + build_answer("").strip()
            )
        #print(model_inputs[0])
        # break

        model_outputs = forward_model(
            prompt=model_inputs[0],
            model=model,
            tokenizer=tokenizer,
        )
        
        model_outputs = model_outputs[0].split("Risposta breve:")[-1] if "Risposta breve:" in model_outputs[0] else _
        print(f"DOMANDA:\n{model_inputs[0]}\n\nRISPOSTA\n{model_outputs}\nSOLUZIONE\n{answers[0]['text']}\n")#[520:]



        # for model_output, id in zip(model_outputs, ids):
        #     predicted_answers.append(dict(
        #         id=id,
        #         prediction_text=model_output,
        #     ))
        predicted_answers.append(dict(
                id=ids[0],
                prediction_text=model_outputs,
            ))

        for ans, id in zip(answers, ids):
            theoretical_answers.append(dict(
                id=id,
                answers=ans,
            ))
        
    except Exception as e:
        print(e)
        continue

  0%|          | 1/7609 [00:01<2:43:19,  1.29s/it]

tensor([[     2,      2,    106,   1645,    108,   5523,  60520,    959, 125021,
         222809,    748, 153202,    526,  24702,   1749,  95962, 235265,   1800,
          28973, 200511,   4039,  29398,    751, 186479,   8097,  95962,   2572,
           2023,    487, 206214,   7809, 153202, 235265,    108,   1739,  18470,
         235292,    108, 235301, 235303,  43445, 235269,    683,  28551, 235269,
            683,  89320, 235269,   1800,   2399, 235704, 235269,   1800,  66823,
            526,    533, 235303,  43445,   8628,  47511,    496,  85104, 128431,
            751,  16706,  29219,   1177,  14451,  88892,    842,   9173,  19202,
          43896,   2590,   2287,  13113, 116261,    751,   1622,   3459,   3922,
           7249, 235248, 235284, 235276, 235274, 235304, 235289,    683, 111025,
         122240, 235269,    683,  48732,    534,   1820, 235269,  52676, 235269,
          66280,    526, 106484,  23758,  42804,  19543, 235289,    637, 203016,
         235269,  28076,    

  0%|          | 2/7609 [00:02<2:26:35,  1.16s/it]

tensor([[     2,      2,    106,   1645,    108,   5523,  60520,    959, 125021,
         222809,    748, 153202,    526,  24702,   1749,  95962, 235265,   1800,
          28973, 200511,   4039,  29398,    751, 186479,   8097,  95962,   2572,
           2023,    487, 206214,   7809, 153202, 235265,    108,   1739,  18470,
         235292,    108,   2841,  28551,    661,   6316,   1817,  42933,  17753,
           7249, 235248, 235274, 235315, 235308, 235304, 235265,  16575, 235248,
         235274, 235315, 235308, 235318, 235269,   1419, 143815,  46728, 235782,
          17909, 235269,  25496, 235703,    476,   6316,    895,    683,   4094,
          13322,    576,  59856,  12947,    578,  15171,  12720,    751, 137186,
           1800,  27743,  22906,   4748,  44636,  38403,  33995, 235265,  17753,
         235269,    526, 119814,  90152,   1234,  66684, 235782,   1800,  27743,
            751,   3402, 102250,   2063,  54951,   4748,   7467,  21405, 132637,
         235265,  16575, 235

  0%|          | 3/7609 [00:04<3:38:32,  1.72s/it]

...

tensor([[     2,      2,    106,   1645,    108,   5523,  60520,    959, 125021,
         222809,    748, 153202,    526,  24702,   1749,  95962, 235265,   1800,
          28973, 200511,   4039,  29398,    751, 186479,   8097,  95962,   2572,
           2023,    487, 206214,   7809, 153202, 235265,    108,   1739,  18470,
         235292,    108,   6870,   1142,  13207,   8022,    575,  11557,    751,
          60289, 235269,  11909,    576,   8186,    591, 235274, 235315, 235315,
         235321,    823,  36215,  19335,    852,    748, 202839,  13113,   7682,
          64435, 235265,   5478,  10642,   1817,    632,   1800, 222991,    751,
         137738,   1820,   7130,  97970,    526,    661, 148874,  91159,    748,
         168518,   1265,   2688,  11085,   6316,    654,  48762,  11877,    632,
           2463,    526,   7095, 148361,   2581, 235265,   4626,   2708,   5252,
          87637,  60334,   7600,  40855, 107660, 235782,   1265,  29398,   1265,
          60289,   2041, 235

  0%|          | 19/7609 [00:28<2:59:20,  1.42s/it]

tensor([[     2,      2,    106,   1645,    108,   5523,  60520,    959, 125021,
         222809,    748, 153202,    526,  24702,   1749,  95962, 235265,   1800,
          28973, 200511,   4039,  29398,    751, 186479,   8097,  95962,   2572,
           2023,    487, 206214,   7809, 153202, 235265,    108,   1739,  18470,
         235292,    108,   6870,  22906,    875,  41141,   2170,    748,  86457,
           7809, 166534,  58379,   1800,  17358, 100709,    751,  74173,  48936,
          10422,  19335, 235269,    718,   1800, 166534,  21829, 109537,    654,
            742,  71538,  74173,    591,    484,  66015,   1800,   4817,    539,
          75168,    664, 187883, 229441,   4764,   1388,    632,   1749,  47802,
          11671,    751,   3402,   1390,   3423,    575,  24200,   7467, 235265,
          42463,    709,   3402,   2270,   1177, 166534,   2173,  26160,  37348,
            476,  76346,    895,   1749,   1423,    751,  21818,  12696, 235269,
           1800,  22906,  21

  0%|          | 19/7609 [00:28<3:11:15,  1.51s/it]


KeyboardInterrupt: 

In [ ]:
import os
import json



# Create the directory if it doesn't exist
os.makedirs('./cache', exist_ok=True)
with open(f"./cache/generated-squad-{model_name}.json", "w") as f:
    json.dump(dict(
        predicted_answers=predicted_answers,
        theoretical_answers=theoretical_answers,
    ), f, indent=4)

In [ ]:
import json
with open(f"./cache/generated-squad-{model_name}.json", "r") as f:
    data = json.load(f)

print(len(data["predicted_answers"]))

In [ ]:
import evaluate

predicted_answers = data["predicted_answers"]
theoretical_answers = data["theoretical_answers"]

metric = evaluate.load("squad")
results = metric.compute(predictions=predicted_answers, references=theoretical_answers)

import time


print("=== REPORT ===")
print("current date:", time.strftime("%d/%m/%Y %H:%M:%S"))
print("Dataset: SQuAD-it")
print("Model:", model_name)
print(results)
print("==========================")